# Analysis for CSC3050 Project 3 Part 1.

## 1.1. Required Environment:

- python interpreter： `Python 3.11` or above

- packages:

  - `Numpy`

  - `Jupyter Notebook`

  - `matplotlib`
  
  - `pandas`

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

## 1.2. CSV loading and visualization 

In [2]:
trace1 = pd.read_csv("./cache-trace/trace1.trace.csv")
trace2 = pd.read_csv("./cache-trace/trace2.trace.csv")

In [3]:
trace1

,cacheSize,blockSize,associativity,writeBack,writeAllocate,missRate,totalCycles
0,4096,32,2,1,1,0.371203,977649
1,4096,32,2,1,0,0.423755,1011873
2,4096,32,2,0,1,0.371203,1156185
3,4096,32,2,0,0,0.423755,1192217
4,4096,32,4,1,1,0.365649,967101
...,...,...,...,...,...,...,...
391,1048576,256,16,0,0,0.008955,653088
392,1048576,256,32,1,1,0.008241,246030
393,1048576,256,32,1,0,0.008955,247192
394,1048576,256,32,0,1,0.008241,652742


In [4]:
trace2

,cacheSize,blockSize,associativity,writeBack,writeAllocate,missRate,totalCycles
0,4096,32,2,1,1,0.226746,519584
1,4096,32,2,1,0,0.453449,679983
2,4096,32,2,0,1,0.226746,716896
3,4096,32,2,0,0,0.453449,679991
4,4096,32,4,1,1,0.226746,519584
...,...,...,...,...,...,...,...
116,16384,128,2,1,1,0.056723,251872
117,16384,128,2,1,0,0.340082,550698
118,16384,128,2,0,1,0.056723,596888
119,16384,128,2,0,0,0.340082,550746
